## Import modules

In [1]:
import pandas as pd
import geopandas as gpd


import html2text
import nltk
import folium

import os
import math


## Define Functions

Define function to debug

In [2]:
import inspect
import re

def prinfo(*args, **kwargs):
    frame = inspect.currentframe().f_back
    s = inspect.getframeinfo(frame).code_context[0]
    r = re.search(r"\((.*)\)", s).group(1)
    vnames = r.split(", ")

    if 'newline' in kwargs:
        newlinestring = "\n" if kwargs['newline'] else ""
    else:
        newlinestring = ""

    for i, (var, val) in enumerate(zip(vnames, args)):
        print(f"{var} = {newlinestring}{val}")


Define function to give information about the progress of reading the csv

In [3]:
def read_csv_in_chunks(path, n_lines, **read_params):
    if 'chunksize' not in read_params or read_params['chunksize'] < 1:
        read_params['chunksize'] = 80000

    chunks = [0] * math.ceil(n_lines / read_params['chunksize'])

    for i, chunk in enumerate(pd.read_csv(path, **read_params)):
        percent = min(((i + 1) * read_params['chunksize'] / n_lines) * 100, 100.0)
        print("#" * int(percent), f"{percent:.2f}%", end='\r', flush=True)
        chunks[i] = chunk[chunk[nameCol].notnull()]
    
    print()
    print("Now concatenating chunks...")
    df = pd.concat(chunks, axis=0)
    del chunks
    print("Finished!")
    return df

## Import Stuff

### Gazetteer

In [4]:
gaz = 'nga'

if gaz == "nga":
    gazetteerpath = "data/nga/countries_administrative.csv"
    # indexOfLat = 4
    # indexOfLon = 5
    indexOfLat = 1
    indexOfLon = 2
    n_lines =  484618 #for nga administrative
    # n_lines =  284485 #for nga administrative approved
    # n_lines = 7866485 #for nga administrative populated
    nameCol = "SORT_NAME_RO" # for nga

elif gaz == "geonames":
    gazetteerpath = "data/geonames/allCountries_cleaned.csv"
    indexOfLat = 2
    indexOfLon = 3
    n_lines = 6974472 # for allCountries_cleaned.csv
    # n_lines = 2079830 # for allCountries_AT.csv
    nameCol = "name"
    


Read gazetter data (csv) and save placenames in a list

In [5]:
if gaz == "countries":
    countrynames = pd.read_csv("data/geodict_github/countrynames.csv", names=["short", "long"])
    countrypositions = pd.read_csv("data/geodict_github/countrypositions.csv", names=["short", "lat", "lon"])

    df_gazetteer = countrynames.merge(countrypositions, on="short")
    df_gazetteer.long = df_gazetteer.long.str.strip()
    indexOfLat = 2
    indexOfLon = 3
    n_lines = 240
    nameCol = "long"
    
else:
    # read in the gazetter csv
    df_gazetteer = read_csv_in_chunks(
        path = gazetteerpath,
        n_lines = n_lines,
        low_memory=False)



#################################################################################################### 100.00%
Now concatenating chunks...
Finished!


### Textfile

Import textfile or call the html2text function to extract the text from a given url and save it for next use

In [6]:
textfile = "data/texts/aid_wiki.txt"

url = 'https://www.theguardian.com/global-development/2021/dec/21/uk-accused-of-abandoning-worlds-poor-as-aid-turned-into-colonial-investment'
text_save_path = f"data/texts/autosave/lastText_{url[-20:]}"
# url = "https://www.theguardian.com/world/2021/oct/21/cuts-to-overseas-aid-thwart-uk-efforts-to-fight-covid-pandemic"

if textfile:
    with open(textfile, "r") as raw_text:
        text = raw_text.read()

else:
    if not os.path.exists(text_save_path):
        allWords = html2text.html2text(url=url).split(" ")

        with open(text_save_path, "w") as lt:
            lt.write(" ".join(allWords))

    else:
        with open(text_save_path, "r") as lt:
            text = lt.read()





## Matching

Extract places from text using nltk

In [23]:
tokenized = nltk.word_tokenize(text)
tree = nltk.ne_chunk(nltk.pos_tag(tokenized))

places = [
    " ".join(i[0] for i in t)
    for t in tree
    if hasattr(t, "label") and t.label() == "GPE"
]

stemmer = nltk.stem.PorterStemmer()
lemmatizer = nltk.stem.WordNetLemmatizer()
places = [lemmatizer.lemmatize(word).upper() for word in places]
prinfo(sorted(places)[:10])

sorted(places)[:10] = ['ADE', 'ADRIAN', 'AFGHANISTAN', 'AFGHANISTAN', 'AFGHANISTAN', 'AFRICA', 'AFRICA', 'AFRICA', 'AFRICA', 'AFRICA']


Create Dictionary with placenames as keys and dictionaries with the counts as values.

In [24]:
d = {}
len_places = len(places)

for i, place in enumerate(places):
    if place in d.keys():
        d[place]["count"] += 1
    else:
        d[place] = {"count": 1}

Fill the dictionary with the coordinates of the placenames.

In [25]:
# add column with lemmatized placenames two compare them to lematized placenames of the text
df_gazetteer["lemma_placenames"] = df_gazetteer[nameCol].apply(lambda place: lemmatizer.lemmatize(place))
df_gazetteer["stem_placenames"] = df_gazetteer[nameCol].apply(lambda place: stemmer.stem(place))

lenKeys = len(d)
logInterval = 1 #int(round(lenKeys/20, 0))

failed_places = []

for i, (placename, attributes) in enumerate(d.items()):
    try:
        tmp_df_values = df_gazetteer.query("lemma_placenames == @placename").values[0]
        # tmp_df_values = df_gazetteer[df_gazetteer["SORT_NAME_RO"] == "MAFIKENG"].values[0]
        attributes["name"] = tmp_df_values[7]
        attributes["lat"] = tmp_df_values[indexOfLat]
        attributes["lon"] = tmp_df_values[indexOfLon]
        
    except IndexError as e:
        failed_places.append(placename)

    # give feedback to progress
    
    if (i % logInterval == 0 and i > 0) or i+1 == lenKeys:
        print(f"{i+1} of {lenKeys} ({round((i/lenKeys)*100, 1)}%)", end='\r')
    

for i, (place, attributes) in enumerate(d.items()):
    if i < 10:
        print((place, attributes))

('AID', {'count': 1})
('ADE', {'count': 1, 'name': 'Adé', 'lat': 12.456154, 'lon': 22.240731})
('AIDE', {'count': 1})
('WIKIPEDIA', {'count': 2})
('HUMANITARIAN', {'count': 8})
('UNITED STATES', {'count': 14})
('CHENERY', {'count': 1})
('STROUT', {'count': 1})
('CHINA', {'count': 5, 'name': 'China', 'lat': 35.0, 'lon': 105.0})
('INDIA', {'count': 1, 'name': 'India', 'lat': 22.0, 'lon': 79.0})


Catch information about nonfound placenames and delete them from the dictionary

In [26]:
num_fails = len(failed_places)
for fail in failed_places:
    del d[fail]

## Visualization

Create a geodataframe with the matched placenames

In [27]:
df = pd.DataFrame(
    [ list(attributes.values()) + [place] for place, attributes in d.items()],
    columns=["count", "name", "lat", "lon", "stemname"])

# df["count"] = df["count"].astype('category')



In [28]:

geo = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.lon, df.lat),
    crs=4326
)

# geojsonname = textfile[textfile.find("/")+1:textfile.find(".")][6:]
# geo.to_file(f"data/geodataframes/{geojsonname}.geojson", driver='GeoJSON')

geo.head()

,count,name,lat,lon,stemname,geometry
0,1,Adé,12.456154,22.240731,ADE,POINT (22.24073 12.45615)
1,5,China,35.000000,105.000000,CHINA,POINT (105.00000 35.00000)
2,1,India,22.000000,79.000000,INDIA,POINT (79.00000 22.00000)
3,1,Turkey,39.000000,35.000000,TURKEY,POINT (35.00000 39.00000)
4,3,Germany,51.500000,10.500000,GERMANY,POINT (10.50000 51.50000)


In [29]:
import random
random.seed(1)

colordic = {}
for i in range(geo["count"].max()):
    colordic[i+1] = "#%06x" % random.randint(0, 0xFFFFFF)
colordic = {
    1: '#440154',
    2: '#3b528b',
    3: '#21918c',
    4: '#5ec962',
    5: '#fde725'
}
print(colordic)

color_list = [colordic[count] for count in geo["count"]]

{1: '#440154', 2: '#3b528b', 3: '#21918c', 4: '#5ec962', 5: '#fde725'}


In [30]:
pointMap = folium.Figure(width='35%')
gpd.GeoSeries.explore(
    geo,
    color=color_list,
    max_bounds=True,
    tiles="Open Street Map", # "Stamen Watercolor",
    marker_type='circle',
    marker_kwds={
        'radius': 50000,
        'fill': True}).add_to(pointMap)

# folium.TileLayer(tiles='stamen watercolor', name="Watercolor").add_to(pointMap)

pointMap

In [31]:
from folium import plugins


# create map
heatMap = folium.Figure(width='50%')
heatMap = folium.Map(
    location=[15, 30], zoom_start=2, tiles=None).add_to(heatMap)

# add tiles
folium.TileLayer(tiles='Cartodb dark_matter', name="Dark").add_to(heatMap)
folium.TileLayer(tiles='stamen watercolor', name="Watercolor").add_to(heatMap)

# add points and markercluster
points = folium.FeatureGroup(name="Points", show=True).add_to(heatMap)
cluster = plugins.MarkerCluster(name="Cluster", show=False).add_to(heatMap)

for place, attributes in d.items():

    coordinates = (attributes['lat'], attributes['lon'])

    html = f'''
    <strong>Name:</strong> &emsp;&emsp;&emsp;&emsp;&emsp;{attributes['name']}<br/>
    <strong>Stemmed Name:</strong>&emsp;{place}<br/>
    <strong>Count:</strong>&emsp;&emsp;&emsp;&emsp;&emsp;{attributes['count']}'''


    iframe = folium.IFrame(
        html,
        width=300,
        height=70)

    popup = folium.Popup(iframe)

    folium.Circle(coordinates).add_to(cluster)
    folium.Circle(
        location=coordinates,
        popup=popup,
        radius=attributes['count'] * 50000,
        fill=True,
        color=colordic[attributes['count']]
    ).add_to(points)

# extract coordinate of geodataframe
coordinates = [[point.xy[1][0], point.xy[0][0]] for point in geo.geometry]

# add heatmap
plugins.HeatMap(
    name='HeatMap',
    data=coordinates,
    min_opacity=0.3,
    show=False
).add_to(heatMap)

# add layercontrol
folium.LayerControl(collapsed=False).add_to(heatMap)


heatMap
